# Handwriting Detect 手写识别

Demo1 Homework

Tastror

## 一、开发环境

gpu 相关

- 显卡为 NVIDIA 公司产品，在 帮助 - 系统信息 中查看 GPU 支持的最高 CUDA 版本为 11.7。
- 在 pytorch 官网（2023/3/21）中支持的 CUDA 版本为 11.7 或 11.8

pytorch 相关

```shell
conda create --name py311_torch python=3.11
conda activate py311_torch
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117  # in 2023/3/21, it is 2.0.0
```

<div align="center">

| pytorch 版本 | Python 版本 | cuDNN | CUDA | Windows 版本 |
| ---- | ---- | ---- | ---- | ---- |
| pytorch-2.0.0 | Any (3.11 is used) | Any (8.8.1 is used) | 11.7 | 11 |

</div>

为了后面的画图方便，我们同时安装 matplotlib。

```shell
pip install matplotlib  # in 2023/3/21, it is 3.7.1
```

以及安装 Demo1 中会用到的库

```shell
pip install scipy  # in 2023/3/21, it is 1.10.1
pip install opencv-contrib-python  # in 2023/3/21, it is 4.7.0.72
pip install kornia  # in 2023/3/21, it is 0.6.10
```

接着查看一下 GPU 是否可用

In [1]:
import torch
print(torch.cuda.is_available())

True


可用，说明环境搭建成功，并且尝试运行 Demo1 中的代码，没有问题。

## 二、样例测试与分析

首先直接运行，看看结果。

运行的最后几行输出如图。

```plaintext
Start Train
Epoch 300/300:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 92/93 [00:00<00:00, 259.91it/s, acc=1, loss=3.25e-7]Start test
Epoch 300/300: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 260.92it/s, acc=1, loss=3.25e-7] 
Epoch 300/300: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 497.29it/s, acc=0.726]
```

训练集上的准确率为 100%，而测试集上的数据只有 72.6%，很明显过拟合了。

我们从源数据和模型两个方面进行分析。

### （一）源数据

对于 Demo1 数据集，其有两部分组成：train 和 test，使用 scipy 读取后可用看到其中的具体数据长度。

```python
self.data = scio.loadmat(path)
self.data = self.data['train'] if train else self.data['test']
self.shape = np.shape(self.data)
self.data_shape = self.shape[:2]
self.image_shape = self.shape[2:]
self.length = self.data_shape[0] * self.data_shape[1]

print('length of dataset is', self.length, end=", ")
print('shape of dataset is', self.data_shape, end=", ")
print('shape of image is', self.image_shape)
```

输出为

```plaintext
length of dataset is 3000, shape of dataset is (200, 15), shape of image is (28, 28)  # training dataset
length of dataset is 1000, shape of dataset is (200, 5), shape of image is (28, 28)  # test dataset
```

也就是训练集有 200 个标签，每个标签下有 15 张图片。图片的大小为 28 * 28，和手写数字类似。

很明显，这个图片的数据量是很小的（MNIST 的训练集有 60,000 张，也就是每个标签下有 6000 张图片），所以在训练之前，我们需要进行一下数据增强。

```python
# 反色预处理
image = 255 - np.array(image)

# 增强
if augmentation_type == 0:
    pass
elif augmentation_type == 1:
    if index % 2 == 0:
        image = np.roll(image, 3, axis=0)  # 平移操作
    else:
        image = np.roll(image, -3, axis=0)  # 平移操作
elif augmentation_type == 2:
    if index % 2 == 0:
        image = np.roll(image, 3, axis=1)  # 平移操作
    else:
        image = np.roll(image, -3, axis=1)  # 平移操作
elif augmentation_type == 3:
    # 旋转操作
    image = Image.fromarray(image)
    image = image.rotate(10)
    image = np.array(image)
elif augmentation_type == 4:
    # 旋转操作
    image = Image.fromarray(image)
    image = image.rotate(-10)
    image = np.array(image)
elif augmentation_type == 5:
    # 缩放操作
    image = Image.fromarray(image)
    image = image.resize((24, 24), resample=Image.BILINEAR)
    image = np.array(image)
    image = np.pad(image, (2, 2), 'constant', constant_values=0)
elif augmentation_type == 6:
    # 缩放操作
    image = Image.fromarray(image)
    image = image.resize((32, 32), resample=Image.BILINEAR)
    image = np.array(image)
    image = image[2:30, 2:30]
```

这里使用基本的平移与缩放进行增强（字符不能旋转），将数据量变为了原来的七倍。（具体代码见 dataloader.py）

最终在 50 个 epoch 的训练下（损失函数已收敛），在**原模型**下的结果为

```plaintext
Epoch 49/50, Loss: 0.0219, Acc: 96.3887
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 657/657 [00:03<00:00, 165.15it/s] 
Epoch 50/50, Loss: 0.0198, Acc: 96.4500
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 271.03it/s]
Test Accuracy: 0.7730
```

测试集上的准确率为 77.3%，比原来高。

### （二）模型

这里我选择添加一层卷积层，并且由于测试集和训练集的准确率相差太大，所以我添加了 dropout。

```plaintext
Epoch 49/50, Loss: 0.0190, Acc: 95.7554
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 657/657 [00:03<00:00, 173.10it/s] 
Epoch 50/50, Loss: 0.0368, Acc: 95.8160
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 312.85it/s]
Test Accuracy: 0.8380
```

测试集上的准确率为 83.8%，比原来高。

经过多次测试，目前表现最优的模型能达到测试集上 90.9% 的准确率。代码如下。

In [ ]:
import torch.nn as nn


# 定义模型
class Net(nn.Module):
    def __init__(self, label_num: int = 200):
        super(Net, self).__init__()
        self.conv_features_1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.2),
        )

        self.conv_features_2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.2),
        )

        self.conv_features_3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),
        )

        self.conv_features_4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),
        )

        self.classifier = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, label_num),
        )
        
    def forward(self, x):
        x = self.conv_features_1(x)
        x = self.conv_features_2(x)
        x = self.conv_features_3(x)
        x = self.conv_features_4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


在不同数据集下的测试结果为（epoch = 196）

| 数据集 | 准确率 |
| --- | --- |
| 训练集 | 98.37% |
| 增强训练集（训练时使用） | 97.69% |
| 测试集 | 91.00% |
| 增强测试集 | 88.36% |

这里的几层的 dropout 是延续前面测试三层卷积的 0.2 0.2 0.25 的 Dropout，它们起到了很好的防过拟合的作用（经测试，0.2 0.2 0.2 就没有这么好的效果了，容易过拟合）。

因此这里的四层卷积使用了 0.2 0.2 0.25 0.25 的 Dropout，结果完全不会过拟合。（在增强数据集上）

后面使用了更多东西，让其效果达到了 95%。自己看去吧。